In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np

def load_images_and_masks(data_dir, mask_dir):
    images = []
    masks = []
    file_names = sorted(os.listdir(data_dir))
    for file in file_names:
        if file.endswith('.png'):
            img_path = os.path.join(data_dir, file)
            mask_path = os.path.join(mask_dir, file)
            try:
                img = load_img(img_path)  # Carrega a imagem no formato padrão
                img = img_to_array(img) / 255.0  # Normaliza a imagem para o intervalo [0, 1]
                mask = load_img(mask_path, color_mode='grayscale')  # Carrega a máscara como grayscale
                mask = img_to_array(mask) / 255.0  # Normaliza a máscara
                mask = (mask > 0.5).astype(np.float32)  # Binariza a máscara

                if img.shape == (1200, 1200, 3) and mask.shape == (1200, 1200, 1):
                    images.append(img)
                    masks.append(mask[:, :, 0])  # Garante que a máscara seja um array 2D
                else:
                    print(f"Descartado por dimensões incorretas: {file}")
            except Exception as e:
                print(f"Erro ao carregar {file}: {e}")

    return np.array(images), np.array(masks)

# Caminhos para os diretórios de imagens e máscaras
data_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli/rgbs'
mask_dir = '/content/drive/MyDrive/modulo10/data/dataset_inteli/masks'

# Carregar imagens e máscaras
images, masks = load_images_and_masks(data_dir, mask_dir)
print(f"Número de imagens carregadas: {len(images)}")
print(f"Número de máscaras carregadas: {len(masks)}")


In [ ]:
images, masks = load_images_and_masks(data_dir, mask_dir)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.1, random_state=42)

In [ ]:
# Função para criar o modelo U-Net
def unet(input_size=(1200, 1200, 3)):
    inputs = Input(input_size)
    # Contração
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Expansão
    u6 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c2)
    u6 = concatenate([u6, c1])
    c6 = Conv2D(16, (3, 3), activation='relu', padding='same')(u6)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(16, (3, 3), activation='relu', padding='same')(c6)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c6)
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Criar o modelo
model = unet(input_size=(1200, 1200, 3))

In [ ]:
# Treinar o modelo
results = model.fit(X_train, y_train, validation_split=0.1, batch_size=2, epochs=50)


In [ ]:
# Avaliar o modelo
model.evaluate(X_test, y_test)

In [ ]:
def plot_results(X, y, model, ix=None):
    """Plotar a imagem, a máscara real e a máscara predita.

    Args:
    - X: array de imagens de entrada.
    - y: array de máscaras reais.
    - model: modelo treinado U-Net.
    - ix: índice da imagem a ser plotada. Se None, seleciona uma imagem aleatoriamente.
    """
    if ix is None:
        ix = np.random.randint(0, len(X))

    fig, ax = plt.subplots(1, 3, figsize=(20, 10))
    ax[0].imshow(X[ix], cmap='gray')
    ax[0].title.set_text('Imagem Original')
    ax[0].axis('off')

    ax[1].imshow(y[ix].squeeze(), cmap='gray')  # Squeeze para remover dimensões extras se houver
    ax[1].title.set_text('Máscara Real')
    ax[1].axis('off')

    # Fazer a predição usando o modelo
    pred = model.predict(X[ix:ix+1])
    pred = (pred > 0.5).astype(np.float32)  # Binarizar a predição

    ax[2].imshow(pred.squeeze(), cmap='gray')  # Squeeze para garantir que está em 2D
    ax[2].title.set_text('Máscara Predita')
    ax[2].axis('off')

    plt.show()


In [ ]:
# Exemplo de como plotar uma imagem específica
index = None  # Escolha um índice ou deixe None para aleatório
plot_results(X_test, y_test, model, ix=index)
